In [1]:
import tensorflow
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from tensorflow.keras.utils import to_categorical
from keras import regularizers
import tensorflow as tf

tf.config.run_functions_eagerly(True)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
data = pd.read_csv('data.csv', index_col = 'row_id')

## Preprocessing

In [5]:
encoder = LabelEncoder()
data['target'] = encoder.fit_transform(data['target'])

In [6]:
train, test = train_test_split(data,
                            test_size=0.2,
                            stratify=data['target'],
                            random_state=42)

In [7]:
train2 = pd.DataFrame(
    [list(tup) for tup in train.value_counts().index.values], 
    columns=train.columns
)

In [8]:
train2['sample_weight'] = train.value_counts().values

In [9]:
Xtrain = train2.drop(['target','sample_weight'],axis=1)
ytrain = train2['target']
class_weights = train2['sample_weight']

In [10]:
Xtest = test.drop('target',axis=1)
ytest = test['target']

In [11]:
print("Number of values in train:",Xtrain.shape[0])
print("Number of values in test:",Xtest.shape[0])

Number of values in train: 104996
Number of values in test: 40000


## Number of hidden layers

In [12]:
ytrain2 = to_categorical(ytrain, num_classes=10)

In [14]:
# Model to test 1 hidden layer

input_seq = Input(shape=Xtrain.shape[1])
hidden = Dense(143,activation='tanh')(input_seq)
output_seq = Dense(10,activation='softmax')(hidden)

model = Model(input_seq,output_seq)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Metal device set to: Apple M1


2022-04-16 09:54:49.782677: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-16 09:54:49.783163: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
model.fit(Xtrain,ytrain2,sample_weight=class_weights, epochs=10, verbose=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10


/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2022-04-16 09:55:27.121124: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3282/3282 [==============================] - 45s 13ms/step - loss: 2.0124 - accuracy: 0.5640
Epoch 2/10
3282/3282 [==============================] - 42s 13ms/step - loss: 1.3567 - accuracy: 0.7660
Epoch 3/10
3282/3282 [==============================] - 43s 13ms/step - loss: 1.1671 - accuracy: 0.8027
Epoch 4/10
3282/3282 [==============================] - 42s 13ms/step - loss: 1.0784 - accuracy: 0.8249
Epoch 5/10
3282/3282 [==============================] - 42s 13ms/step - loss: 1.0236 - accuracy: 0.8393
Epoch 6/10
3282/3282 [==============================] - 42s 13ms/step - loss: 0.9855 - accuracy: 0.8479
Epoch 7/10
3282/3282 [==============================] - 43s 13ms/step - loss: 0.9563 - accuracy: 0.8554
Epoch 8/10
3282/3282 [==============================] - 43s 13ms/step - loss: 0.9339 - accuracy: 0.8607
Epoch 9/10
3282/3282 [==============================] - 43s 13ms/step - loss: 0.9154 - accuracy: 0.8636
Epoch 10/10
3282/3282 [==============================] - 43s 13ms/step - lo

In [16]:
ypred_train = np.argmax(model.predict(Xtrain),axis=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [17]:
ypred_test = np.argmax(model.predict(Xtest),axis=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [18]:
print("Training Accuracy:\t",accuracy_score(ytrain,ypred_train))
print("Testing Accuracy:\t",accuracy_score(ytest,ypred_test))

Training Accuracy:	 0.8683949864756753
Testing Accuracy:	 0.834225


In [19]:
# Model to test 2 hidden layers

input_seq = Input(shape=Xtrain.shape[1])
hidden = Dense(200, activation='tanh')(input_seq)
hidden2 = Dense(105,activation='tanh')(hidden)
output_seq = Dense(10,activation='softmax')(hidden2)

model = Model(input_seq,output_seq)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(Xtrain,ytrain2,sample_weight=class_weights, epochs=10, verbose=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
   4/3282 [..............................] - ETA: 54s - loss: 3.5767 - accuracy: 0.1250 

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3282/3282 [==============================] - 47s 14ms/step - loss: 1.5389 - accuracy: 0.6760
Epoch 2/10
3282/3282 [==============================] - 47s 14ms/step - loss: 1.0883 - accuracy: 0.8211
Epoch 3/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.9878 - accuracy: 0.8461
Epoch 4/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.8715 - accuracy: 0.8658
Epoch 5/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.7600 - accuracy: 0.8830
Epoch 6/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.6932 - accuracy: 0.8943
Epoch 7/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.6490 - accuracy: 0.8995
Epoch 8/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.6157 - accuracy: 0.9049
Epoch 9/10
3282/3282 [==============================] - 47s 14ms/step - loss: 0.5866 - accuracy: 0.9081
Epoch 10/10
3282/3282 [==============================] - 47s 14ms/step - lo

In [21]:
ypred_train = np.argmax(model.predict(Xtrain),axis=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [22]:
ypred_test = np.argmax(model.predict(Xtest),axis=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [23]:
print("Training Accuracy:\t",accuracy_score(ytrain,ypred_train))
print("Testing Accuracy:\t",accuracy_score(ytest,ypred_test))

Training Accuracy:	 0.9037106175473352
Testing Accuracy:	 0.8649


In [24]:
# Model to test 3 hidden layers

input_seq = Input(shape=Xtrain.shape[1])
hidden = Dense(225, activation='tanh')(input_seq)
hidden2 = Dense(153,activation='tanh')(hidden)
hidden3 = Dense(82,activation='tanh')(hidden2)
output_seq = Dense(10,activation='softmax')(hidden3)

model = Model(input_seq,output_seq)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(Xtrain,ytrain2,sample_weight=class_weights, epochs=10, verbose=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
3282/3282 [==============================] - 50s 15ms/step - loss: 1.4253 - accuracy: 0.6988
Epoch 2/10
3282/3282 [==============================] - 51s 15ms/step - loss: 0.8790 - accuracy: 0.8581
Epoch 3/10
3282/3282 [==============================] - 51s 15ms/step - loss: 0.7182 - accuracy: 0.8830
Epoch 4/10
3282/3282 [==============================] - 50s 15ms/step - loss: 0.6512 - accuracy: 0.8929
Epoch 5/10
3282/3282 [==============================] - 50s 15ms/step - loss: 0.6049 - accuracy: 0.8998
Epoch 6/10
3282/3282 [==============================] - 51s 15ms/step - loss: 0.5754 - accuracy: 0.9037
Epoch 7/10
3282/3282 [==============================] - 51s 15ms/step - loss: 0.5600 - accuracy: 0.9052
Epoch 8/10
3282/3282 [==============================] - 51s 15ms/step - loss: 0.5363 - accuracy: 0.9093
Epoch 9/10
3282/3282 [==============================] - 51s 15ms/step - loss: 0.5207 - accuracy: 0.9118
Epoch 10/10
3282/3282 [==============================] - 51s 15m

In [26]:
ypred_train = np.argmax(model.predict(Xtrain),axis=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [27]:
ypred_test = np.argmax(model.predict(Xtest),axis=1)

/Users/himanshu/miniforge3/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [28]:
print("Training Accuracy:\t",accuracy_score(ytrain,ypred_train))
print("Testing Accuracy:\t",accuracy_score(ytest,ypred_test))

Training Accuracy:	 0.9034153682045031
Testing Accuracy:	 0.85265


### Inference

2 layers offer better performance than 1 layer. 3 layers don't offer any significant improvement in performance, but there are indications of more overfitting with 3 layers.